# AIT500 - Course Project Checkpoint 2 (23-Feb-2024)

This is a helper notebook to walk you through using threads for motor control.  This allows the main body of your program to read data from the BMP280 to decide which motors should be activated.

The code below starts with a basic threading class called `BotThread` and walks you through three versions of incremental code development and testing to implement a program that performs the needed functionality derived from the logic flow diagram discussed in class

In [ ]:
import threading
import time
from datetime import datetime as dt

# Assuming these bot control functions exist
def move_forward(perc_pwm):
    print(f"Percent PWM:{perc_pwm}, Bot moving forward")

def turn_right(perc_pwm):
    print(f"Percent PWM:{perc_pwm}, Bot turning right")

def turn_left(perc_pwm):
    print(f"Percent PWM:{perc_pwm}, Bot turning left")

def move_stop(perc_pwm=0):
    print(f"Percent PWM:{perc_pwm}, Bot stopped")


# Thread class for moving the bot
class BotThread(threading.Thread):
  def __init__(self, perc_pwm=0):
    # default to off with 0% PWM
    super().__init__()
    self.action_event = threading.Event()
    self.current_action = move_stop
    self.perc_pwm = perc_pwm

  def run(self):
    while True:
      if self.action_event.is_set():
        self.current_action(self.perc_pwm)
        self.action_event.clear()  # Reset the event after taking the action
      # else:
        # Default action continue previous action
      time.sleep(.1)  # Add delay to simulate movement time

  def update_action(self, new_action, perc_pwm=0):
    self.current_action = new_action
    self.perc_pwm = perc_pwm
    self.action_event.set()

In [ ]:
def read_BMP280():
  return 10

# Work with threads - v1

testing functionality in the `BotThread` class

In [ ]:
perc_pwm = .2
MAX_COUNTER = 100

# Create and start the bot thread
bot_thread = BotThread(perc_pwm=perc_pwm)
bot_thread.start()

try:
  for idx in range(MAX_COUNTER):
    sensor_data = read_BMP280()
    if idx==20: # after 20 seconds, test turn_left
      print(f"1-{idx}")
      # Sensor value below threshold, signal the bot thread to take different action
      bot_thread.update_action(turn_left, 0.2)  # For example, turn left when sensor data is low
    elif idx==40: # after 40 seconds, test turn_left
      print(f"2-{idx}")
      # Sensor value below threshold, signal the bot thread to take different action
      bot_thread.update_action(turn_right, 0.3)  # For example, turn left when sensor data is low
    elif idx==60: # after 60 seconds, test turn_left
      print(f"3-{idx}")
      # Sensor value below threshold, signal the bot thread to take different action
      bot_thread.update_action(move_forward, 0.4)  # For example, turn left when sensor data is low
    elif idx==80: # after 80 seconds, test turn_left
      print(f"4-{idx}")
      # Sensor value below threshold, signal the bot thread to take different action
      bot_thread.update_action(move_stop, 0.5)  # For example, turn left when sensor data is low
    time.sleep(.1)  # Polling delay
except KeyboardInterrupt:
    # Stop the bot and exit cleanly on keyboard interrupt
    bot_thread.update_action(move_stop)
    bot_thread.join()


1-20
Percent PWM:0.2, Bot turning left
2-40
Percent PWM:0.3, Bot turning right
3-60
Percent PWM:0.4, Bot moving forward
4-80
Percent PWM:0.5, Bot stopped


# Work with threads - v2

incorporate max running time

MAX_RUNTIME (in seconds)

In [ ]:
MAX_RUNTIME=5
perc_pwm = .2
MAX_COUNTER = 100

# Create and start the bot thread
bot_thread = BotThread(perc_pwm=perc_pwm)
bot_thread.start()

start_time = dt.now()

try:
  for idx in range(MAX_COUNTER):
    current_time = dt.now()
    if(current_time-start_time).seconds>MAX_RUNTIME:
      break

    sensor_data = read_BMP280()
    if idx==20:
      print(f"1-{idx}")
      # Sensor value below threshold, signal the bot thread to take different action
      bot_thread.update_action(turn_left, 0.2)  # For example, turn left when sensor data is low
    elif idx==40:
      print(f"2-{idx}")
      # Sensor value below threshold, signal the bot thread to take different action
      bot_thread.update_action(turn_right, 0.3)  # For example, turn left when sensor data is low
    elif idx==60:
      print(f"3-{idx}")
      # Sensor value below threshold, signal the bot thread to take different action
      bot_thread.update_action(move_forward, 0.4)  # For example, turn left when sensor data is low
    elif idx==80:
      print(f"4-{idx}")
      # Sensor value below threshold, signal the bot thread to take different action
      bot_thread.update_action(move_stop, 0.4)  # For example, turn left when sensor data is low
    time.sleep(0.1)  # Polling delay
  if(idx<MAX_COUNTER-1):
    print(f"counter at: {idx} / {MAX_COUNTER} - stopped because MAX_RUNTIME seconds triggered")
  else:
    print(f"counter at: {idx} / {MAX_COUNTER} - stopped because for-loop completed")
except KeyboardInterrupt:
    # Stop the bot and exit cleanly on keyboard interrupt
    bot_thread.update_action(move_stop)
    bot_thread.join()


1-20
Percent PWM:0.2, Bot turning left
2-40
Percent PWM:0.3, Bot turning right
counter at: 60 / 100 - stopped because MAX_RUNTIME seconds triggered


# Work with threads - v3

incorporate temperature control thresholds
```
LEFT_TEMP
RIGHT_TEMP
STOP_TEMP
```

In [ ]:
def read_BMP280(idx=0):
    # simulate applying hot or cold to the BMP280 sensor
    if idx<20:
      return 23 # room temperature
    elif (idx>=20)&(idx<40):
      return 35 # hot
    elif (idx>=40)&(idx<60):
      return 28 # down to room temperature
    elif (idx>=60)&(idx<80):
      return 10 # down to cold temperature
    elif idx>=80:
      return 5 # very cold
    else:
      return 23


In [ ]:
MAX_RUNTIME=50
perc_pwm = .2
MAX_COUNTER = 100

LEFT_TEMP = 16
RIGHT_TEMP = 30
STOP_TEMP = 5

# Create and start the bot thread
bot_thread = BotThread(perc_pwm=perc_pwm)
bot_thread.start()

start_time = dt.now()

sensor_data = 23
try:
  for idx in range(MAX_COUNTER):
    current_time = dt.now()
    if(current_time-start_time).seconds>MAX_RUNTIME:
      break

    # simulate applying hot or cold to the BMP280 sensor
    sensor_data = read_BMP280(idx)

    if (sensor_data<=RIGHT_TEMP) & (sensor_data>LEFT_TEMP):
      print(f">1-{idx}, {sensor_data}<", end="")
      # Sensor value in neutral range, move forward
      bot_thread.update_action(move_forward, 0.4)
    elif sensor_data<=STOP_TEMP:
      print(f">2-{idx}, {sensor_data}<", end="")
      # Sensor value below stop threshold, stop
      bot_thread.update_action(move_stop)
    elif (sensor_data>STOP_TEMP) & (sensor_data<=LEFT_TEMP):
      print(f">3-{idx}, {sensor_data}<", end="")
      # Sensor value in left turn range
      bot_thread.update_action(turn_left, 0.2)
    elif sensor_data>=RIGHT_TEMP:
      print(f">4-{idx}, {sensor_data}<", end="")
      # Sensor value in right turn range
      bot_thread.update_action(turn_right, 0.2)

    time.sleep(.5)  # Polling delay

  if(idx<MAX_COUNTER-1):
    print(f"counter at: {idx} / {MAX_COUNTER} - stopped because MAX_RUNTIME seconds triggered")
  else:
    print(f"counter at: {idx} / {MAX_COUNTER} - stopped because for-loop completed")
except KeyboardInterrupt:
    # Stop the bot and exit cleanly on keyboard interrupt
    bot_thread.update_action(move_stop)
    bot_thread.join()


>1-0, 23<Percent PWM:0.4, Bot moving forward
>1-1, 23<Percent PWM:0.4, Bot moving forward
>1-2, 23<Percent PWM:0.4, Bot moving forward
>1-3, 23<Percent PWM:0.4, Bot moving forward
>1-4, 23<Percent PWM:0.4, Bot moving forward
>1-5, 23<Percent PWM:0.4, Bot moving forward
>1-6, 23<Percent PWM:0.4, Bot moving forward
>1-7, 23<Percent PWM:0.4, Bot moving forward
>1-8, 23<Percent PWM:0.4, Bot moving forward
>1-9, 23<Percent PWM:0.4, Bot moving forward
>1-10, 23<Percent PWM:0.4, Bot moving forward
>1-11, 23<Percent PWM:0.4, Bot moving forward
>1-12, 23<Percent PWM:0.4, Bot moving forward
>1-13, 23<Percent PWM:0.4, Bot moving forward
>1-14, 23<Percent PWM:0.4, Bot moving forward
>1-15, 23<Percent PWM:0.4, Bot moving forward
>1-16, 23<Percent PWM:0.4, Bot moving forward
>1-17, 23<Percent PWM:0.4, Bot moving forward
>1-18, 23<Percent PWM:0.4, Bot moving forward
>1-19, 23<Percent PWM:0.4, Bot moving forward
>4-20, 35<Percent PWM:0.2, Bot turning right
>4-21, 35<Percent PWM:0.2, Bot turning right
